https://dacon.io/competitions/official/136/codeshare/1383?page=1&dtype=recent&ptype=pub
<br>를 토대로 코드 작성중
: Facebook prophet 활용 예측 모델

한국의 약 2,000개 상점의 신용카드 거래 내역이 제공<br>
test.csv랑 train.csv의 store_id가 같아도 같은 상점인 것은 아님.
- store_id : 각 파일에서의 상점 고유 번호 (해당기한 내 개업 혹은 폐업한 상점도 있음)
- date : 거래 일자
- time : 거래 시간
- card_id : 카드 번호의 hash 값
- amount : 매출 금액, 음수인 경우 취소 거래
- installments : 할부 개월 수. 일시불은 빈 문자열
- days_of_week : 요일, 월요일이 0, 일요일은 6
- holiday : 1이면 공휴일, 0이면 공휴일 아님

In [1]:
!unzip data01.zip

Archive:  data01.zip
  inflating: submission.csv          
  inflating: test.csv.zip            
  inflating: train.csv.zip           


In [2]:
!unzip train.csv.zip
!unzip test.csv.zip  

Archive:  train.csv.zip
  inflating: train.csv               
   creating: __MACOSX/
  inflating: __MACOSX/._train.csv    
Archive:  test.csv.zip
  inflating: test.csv                
  inflating: __MACOSX/._test.csv     


In [51]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
import pandas as pd
import datetime #requests, 
import os

from fbprophet import Prophet
# Facebook에서 만든 Prophet (시계열 데이터 예측)
import xgboost as xgb
from sklearn.model_selection import train_test_split
import warnings
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')
#한글 깨짐방지
plt.rc('font',family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)

In [52]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("submission.csv")
merge_df = pd.concat([train, test])
merge_df.shape

(3836188, 8)

In [53]:
#전체 날짜의 휴일 요일
all_dates = merge_df.drop_duplicates('date').sort_values('date') #'date'행으로 정렬 후 중복되는 것 삭제
all_dates = all_dates[['date', 'days_of_week', 'holyday']] #날짜, 주차, 휴일
all_dates.reset_index(drop=True, inplace=True) #index reset
all_dates['date'] = pd.to_datetime(all_dates['date']) #별 차이 없음 
all_dates

,date,days_of_week,holyday
0,2016-08-01,0,0
1,2016-08-02,1,0
2,2016-08-03,2,0
3,2016-08-04,3,0
4,2016-08-05,4,0
...,...,...,...
725,2018-07-27,4,0
726,2018-07-28,5,0
727,2018-07-29,6,0
728,2018-07-30,0,0


In [54]:
#휴일만 
holidays = all_dates[all_dates['holyday']>0][['date', 'holyday']]
holidays.columns = ['ds', 'holiday'] #columns 이름 변경
#이 둘은 어디에 쓰이는지 아직 모르겠음
holidays['lower_window'] = 0
holidays['upper_window'] = 0 
holidays.holiday = holidays.holiday.astype('str') #str형으로 변경

#전처리

In [55]:
#store_id와 date가 같은 것들로 묶은 그룹에서 amount 값을 sum() 후 index를 reset
#!!그냥 합산했을 때 음수가 발생하는 경우가 있음!!
d_sum = test.groupby(['date', 'store_id'])['amount'].sum().reset_index() 
d_sum[d_sum['amount']<0].shape

(49, 3)

In [56]:
test_sorted_by_values = test.sort_values(by=[ 'card_id', 'store_id', 'date','time']).reset_index(drop = True)
minus_index = test_sorted_by_values[test_sorted_by_values['amount']<0].index
minus_index.shape

(7943,)

In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 473392 entries, 0 to 473391
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   store_id      473392 non-null  int64  
 1   date          473392 non-null  object 
 2   time          473392 non-null  object 
 3   card_id       473392 non-null  object 
 4   amount        473392 non-null  int64  
 5   installments  1779 non-null    float64
 6   days_of_week  473392 non-null  int64  
 7   holyday       473392 non-null  int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 28.9+ MB


In [58]:
test.loc[5]['amount']

172

In [80]:
test.loc[5]['amount'] == test.loc[5]['amount']

True

In [108]:
'''음수값 처리 조건 : 동일 store_id, card_id이어야하며 날짜는 minus보다 이전 날짜, 금액은 minus와 동일 금액" '''
def del_minus(df):
  card_id_sort = df.sort_values(by=[ 'card_id', 'store_id', 'date','time']).reset_index(drop = True)
  minus_index = card_id_sort[card_id_sort['amount']<0].index
  for i in minus_index:
    del_index = i - 1
    while card_id_sort.iloc[minus_index].amount != card_id_sort.iloc[del_index].amount:
      del_index = del_index - 1
    print(card_id_sort.loc[del_index])
    print(card_id_sort.loc[minus_index])
    card_id_sort.loc[minus_index]['amount'] = 0
    card_id_sort.loc[del_index]['amount'] = 0

In [112]:
test

(473392, 8)

In [154]:
'''음수값 처리 조건 : 동일 store_id, card_id이어야하며 날짜는 minus보다 이전 날짜, 금액은 minus와 동일 금액" '''
#현재 동일 금액으로 처리했을 때 찾지 못하는 값들이 존재하고 있음
def del_minus(df):
  card_id_sort = df.sort_values(by=[ 'card_id', 'store_id', 'date','time']).reset_index(drop = True)
  minus_index = card_id_sort[card_id_sort['amount']<0].index

  for i in minus_index:
    minus_amount = card_id_sort.iloc[i].amount
    range_df = card_id_sort[card_id_sort.index < i] #index 이전 df
    same_df = range_df[range_df.amount.apply(lambda x : x == abs(minus_amount))] 
    #print(same_df)
    result = same_df.tail(n=1)
    #print(result.index, i)
    card_id_sort = card_id_sort.drop(result.index) 
    card_id_sort = card_id_sort.drop(i) 
    #print(card_id_sort.shape)
  
  re_sort_df =  card_id_sort.sort_values(by=['store_id', 'date','time']).reset_index(drop = True)
  return re_sort_df

In [ ]:
new_test = del_minus(test)

,date,store_id,amount
0,2016-08-01,0,2106
1,2016-08-01,2,647
2,2016-08-01,3,199
3,2016-08-01,4,279
4,2016-08-01,6,718
